<a href="https://colab.research.google.com/github/ag4267research1/Solving-Poisson-s-Equation-with-GNO/blob/main/GNOPDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U neuraloperator
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.6/248.6 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 112.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.2/788.2 kB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 54.4 MB/s eta 0:00:00


In [2]:
import torch
import neuralop
import torch.nn as nn
from torch.utils.data import DataLoader

# Import the dataset and processor
from neuralop.data.datasets.nonlinear_poisson import load_nonlinear_poisson_pt

# Import core neural operator components
from neuralop.layers.gno_block import GNOBlock
from neuralop.layers.integral_transform import IntegralTransform
#  Utilities for normalization, training loops, etc.
from neuralop.training import Trainer
from neuralop.utils import count_model_params, count_tensor_params


In [3]:
import matplotlib.pyplot as plt
from torchmetrics import MeanSquaredError
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
!rm -f /content/neuralop_data/nonlinear_poisson.obj
!mkdir -p /content/neuralop_data


In [6]:


# Load a small subset of the dataset (1 training + 1 testing instance)
train_loader, test_loader, data_processor = load_nonlinear_poisson_pt(
    data_path="/content/neuralop_data/nonlinear_poisson.obj",
    n_train=1,
    n_test=1
)


File /content/neuralop_data/nonlinear_poisson.obj already exists. Skipping download.


UnpicklingError: pickle data was truncated

In [ ]:

# Inspect one batch
batch = next(iter(train_loader))

for key, value in batch.items():
    if torch.is_tensor(value):
        print(f"{key:35s} shape={tuple(value.shape)} dtype={value.dtype}")
    else:
        print(f"{key:35s} type={type(value)}")


In [14]:
!mkdir -p /content/neuralop_data
!wget -O /content/neuralop_data/nonlinear_poisson.obj https://zenodo.org/records/15001788/files/nonlinear_poisson.obj


--2025-11-28 18:46:31--  https://zenodo.org/records/15001788/files/nonlinear_poisson.obj
Resolving zenodo.org (zenodo.org)... 188.185.43.153, 188.185.48.75, 137.138.52.235, ...
Connecting to zenodo.org (zenodo.org)|188.185.43.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9608410281 (8.9G) [application/octet-stream]
Saving to: ‘/content/neuralop_data/nonlinear_poisson.obj’

/content/neuralop_d 100%[===================>]   8.95G  18.0MB/s    in 8m 32s  

2025-11-28 18:55:03 (17.9 MB/s) - ‘/content/neuralop_data/nonlinear_poisson.obj’ saved [9608410281/9608410281]

